# Calculations of supercurrents: examples

## The Python conda environment that is used
* We used the following Python environment. See [README.md](README.md) for installation instructions.
```yaml
name: python3
channels:
  - conda-forge
dependencies:
  - python=3.6
  - kwant=1.3
  - holoviews=1.8.1
  - xarray=0.9.6
  - pandas=0.20.3
  - pytables=3.4.2
  - toolz=0.8.2
  - hpc05  # This is only needed if you use a PBS cluster with headnode, as on the TU Delft network
```

In [0]:
import holoviews as hv
import kwant
import numpy as np

# Local imports
import funcs
from funcs import constants, discretized_hamiltonian, get_cuts, add_vlead, hopping_between_cuts

%matplotlib inline
hv.notebook_extension()

def plot_CPR(syst, hopping, params, tol=0.01, max_frequencies=1000):
    phases = np.linspace(-np.pi, np.pi, 51)
    H_0_cache = []
    I = [funcs.current_at_phase(syst, hopping, params, H_0_cache, phase, tol, max_frequencies)
                  for phase in phases]

    return hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'], label='Nummerical CPR')

# 1D

In [0]:
syst, hopping = funcs.make_1d_wire(a=10, L=100, L_sc=100)

params = dict(T=0.06, Delta=0.250, mu=15, k=constants.k,
              hbar=constants.hbar, m_eff=constants.m_eff, current_unit=constants.current_unit, c=constants.c, 
              B_x=0, B_y=0, B_z=0, g=0, mu_B=0, alpha=0, V=lambda x:0)

plot_CPR(syst, hopping, params)

# Simpler way of calculating the super current (but slower)

In [0]:
from funcs import matsubara_frequency

def null_G(syst, params, n):
    en = matsubara_frequency(n, params)
    gf = kwant.greens_function(syst, en, out_leads=[0], in_leads=[0],
                               check_hermiticity=False, params=params)
    return gf.data[::2, ::2]

def current_from_G_0(G_0s, H12, phase, params):
    t = H12 * np.exp(1j * phase)
    dim = t.shape[0]
    I = 0
    for G_0 in G_0s:
        V = np.zeros_like(G_0, dtype=complex)
        v = t - H12
        V[:dim, dim:] = v.T.conj()
        V[dim:, :dim] = v
        gf = np.linalg.solve(np.identity(2*dim) - G_0 @ V, G_0)
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * params['T'] * params['current_unit'] * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I

matsfreqs = 500
G_0s = [null_G(syst, params, n) for n in range(matsfreqs)]
H12 = hopping(syst, params=params)
phases = np.linspace(-np.pi, np.pi, 51)
I = [current_from_G_0(G_0s, H12, phase, params) for phase in phases]
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# Test 2D system and compare with exact equation

In [0]:
a = 1

syst, hopping = funcs.make_2d_test_system(Y=2*a, X=2*a, a=a)
kwant.plot(syst);

params = dict(T=0.0001, hbar=constants.hbar, m=constants.m_eff, 
              Delta=0.001, mu=constants.t*4 / a**2, k=constants.k, current_unit=constants.current_unit, c=constants.c)
kwant.plotter.bands(syst.leads[1], params=params);

# Analytical comparison
phases = np.linspace(-np.pi, np.pi, 51)
N_modes = 2
tanh_part = np.tanh(params['Delta'] / (2 * params['T'] * params['k']) * np.sqrt(1 - np.sin(0.5 * phases)**2))
sum_part = np.sin(phases) / np.sqrt(1 - np.sin(0.5 * phases)**2)
prefactor = constants.eV * (params['Delta'] * constants.meV) / (2 * constants.hbar) # Delta in J
I_analytical = prefactor * N_modes * sum_part * tanh_part * 1e9 # Eq. (35) from arXiv:cond-mat/0406127v2

(plot_CPR(syst, hopping, params, tol=0.001) *
 hv.Curve((phases, I_analytical), kdims=['phase'], vdims=['$I$'], label='Analytical result'))


# 3D test system

In [0]:
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, mu: -mu * sz_
onsite_lead = lambda site, mu, delta: -mu * sz_ + delta * sx_
hops = lambda site1, site2, t: -t * sz_

def make_test_system(X, Y, Z):
    lat = kwant.lattice.general(np.eye(3), norbs=2)
    syst = kwant.Builder()
    syst[(lat(x, y, z) for x in range(X) for y in range(Y) for z in range(Z))] = onsite
    syst[lat.neighbors()] = hops

    cuts = get_cuts(syst, lat)
    syst = add_vlead(syst, lat, *cuts)

    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0, 0)))
    lead[(lat(0, y, z) for y in range(Y) for z in range(Z))] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    hopping = hopping_between_cuts(syst, *cuts)
    
    return syst, hopping

delta = 0.01
params = dict(mu=0., t=1., delta=delta, k=1, current_unit=1, T=delta/100)
syst, hopping = make_test_system(X=3, Y=3, Z=3)
kwant.plot(syst);
plot_CPR(syst, hopping, params)

# 3D system with Discretizer Hamiltonian, squared system

In [0]:
# Test Hamiltonian
syst, hopping = funcs.make_3d_test_system(X=5, Y=3, Z=3, a=1, test_hamiltonian=True)
params = dict(T=0.01, Delta=1, mu=2, t=1, k=1, current_unit=1, c=1)
kwant.plotter.bands(syst.leads[2], params=params)
plot_CPR(syst, hopping, params)

In [0]:
# Full Hamiltonian
syst, hopping = funcs.make_3d_test_system(X=30, Y=30, Z=30, a=10, test_hamiltonian=False)
params = dict(T=1, Delta=0.250, mu=35, B_x=0, B_y=0, B_z=0, g=0, alpha=0, V=lambda x:0, **constants.__dict__)
kwant.plot(syst);
kwant.plotter.bands(syst.leads[2], params=params);
plot_CPR(syst, hopping, params)

# 3D full wire

In [0]:
syst_pars = dict(a=8, angle=0, site_disorder=True, L=80, L_sc=8,
                 phi=135, r1=50, r2=70, shape='circle', with_leads=True,
                 with_shell=True, with_vlead=False, holes=True)

syst, hopping = funcs.make_3d_wire(**syst_pars)

kwant.plot(syst)